In [123]:
from cv2 import reduce
import cv2
# 读取图片
img = cv2.imread("..//image//lena.tif")
# 图片信息
print("图片尺寸:",img.shape)
print("图片数据:",type(img),img)
# 显示图片
cv2.imshow("Lenna", img)
k = cv2.waitKey(0) # waitKey代表读取键盘的输入，0代表一直等待
if k ==27:     # 键盘上Esc键的键值
    cv2.destroyAllWindows()
# 添加文字
cv2.putText(img, "Learn Big Data", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),4)
# 保存图片
cv2.imwrite("img/Lenna_new.png",img)
def mdot(*args):
    return reduce (np.dot,args)
def draw_keypoints(vis,keypoints,color=(0,255,255)):
    for kp in keypoints:
        x,y = kp.pt
        cv2.circle(vis,(int(x),int(y)),2,color)

图片尺寸: (512, 512, 3)
图片数据: <class 'numpy.ndarray'> [[[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [122 148 230]
  [110 130 221]
  [ 90  99 200]]

 [[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [122 148 230]
  [110 130 221]
  [ 90  99 200]]

 [[125 137 226]
  [125 137 226]
  [133 137 223]
  ...
  [122 148 230]
  [110 130 221]
  [ 90  99 200]]

 ...

 [[ 60  18  84]
  [ 60  18  84]
  [ 58  27  92]
  ...
  [ 84  73 173]
  [ 76  68 172]
  [ 79  62 177]]

 [[ 57  22  82]
  [ 57  22  82]
  [ 62  32  96]
  ...
  [ 79  70 179]
  [ 81  71 181]
  [ 81  74 185]]

 [[ 57  22  82]
  [ 57  22  82]
  [ 62  32  96]
  ...
  [ 79  70 179]
  [ 81  71 181]
  [ 81  74 185]]]


In [124]:
import cv2  
import numpy as np  
SZ = 20
CLASS_N = 10
def split2d(img, cell_size, flatten=True):  
    h, w = img.shape[:2]  
    sx, sy = cell_size  
  
    # 确保图像可以被均匀分割，否则需要裁剪或填充图像  
    nx, ny = w // sx, h // sy  # 使用地板除法来确定能够完整分割的块数  
    img_cropped = img[:ny*sy, :nx*sx]  # 裁剪图像以确保可以均匀分割  
  
    cells = [np.hsplit(row, nx) for row in np.vsplit(img_cropped, ny)]  
  
    # 将列表的列表（由vsplit和hsplit产生）转换为一个NumPy数组，  
    # 并可选地将其形状改变为一个三维数组，其中每个子图像都是平坦的  
    cells = np.array(cells)  
    if flatten:  
        cells = cells.reshape(-1, sy, sx)  
  
    return cells  

In [125]:
def load_digits(fn):
    digits_img = cv2.imread(fn,0)
    digits = split2d(digits_img,(SZ,SZ))
    labels = np.repeat(np.arange(CLASS_N),len((digits)/CLASS_N))
    return digits,labels

In [126]:
def deskew(img):
    m = cv2.moments(img)
    if(abs(m['mu02'])) < 1e-2:
        return img.copy()
    skew = m['mu11'] / m['mu02']
    M = np.float32( [[1,skew,-0.5*SZ*skew],[0,1,0]] )
    img = cv2.warpAffine(img, M, (SZ, SZ), flags=cv2.INTER_LINEAR)
    return img

In [127]:
def svmInit(C=12.5,gamma=0.50625):
    model = cv2.ml.SVM_create()
    model.setGamma(gamma)
    model.setC(C)
    model.setKernel(cv2.ml.SVM_RBF)
    model.setType(cv2.ml.SVM_C_SVC)

In [128]:
def process_simple(digits):
    return np.float32(digits).reshape(-1,SZ*SZ) / 255.0

def gethog():
    winSize = (20,20)
    blockSize = (8,8)
    blockStride = (4,4)
    cellSize = (8,8)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0.2
    gammaCoreection = 1
    nlevels = 64
    signedGradient = True
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,   derivAperture,winSigma,histogramNormType,gammaCoreection,nlevels,signedGradient)
    return hog

In [129]:
import numpy as np
from skimage.feature import haar_like_feature
from skimage.transform import integral_image

def get_haar(img):
    integral_image = cv2.integral(img)
    feature_type = 'type-4'
    feature = haar_like_feature(integral_image,0,0,integral_image.shape[1],integral_image.shape[0],feature_type)
    return feature

In [130]:
print("加载本地图片")
digits,labels = load_digits("../image/digits.png")

加载本地图片


In [131]:

from tqdm import tqdm

print('提取每张图片的haar特征向量')
digits_deskewed = list(map(deskew, digits))
haar_features = []  
for img in tqdm(digits_deskewed):  
    haar_features.append(get_haar(img))  
haar_features = np.squeeze(np.array(haar_features)).astype(np.float32)
print(haar_features.shape,haar_features.dtype)
def get_hog():
    """
    :return: 返回一个获得HOG描述符的函数
    """
    winSize = (20, 20)
    blockSize = (8, 8)
    blockStride = (4, 4)
    cellSize = (8, 8)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradient = True

    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma,
                            histogramNormType, L2HysThreshold, gammaCorrection, nlevels, signedGradient)

    return hog
import itertools as it

def grouper(n, iterable, fillvalue=None):
    '''grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx'''
    args = [iter(iterable)] * n
    if PY3:
        output = it.zip_longest(fillvalue=fillvalue, *args)
    else:
        output = it.izip_longest(fillvalue=fillvalue, *args)
    return output
import sys
PY3 = sys.version_info[0] == 3
def mosaic(w, imgs):
    '''Make a grid from images.

    w    -- number of grid columns
    imgs -- images (must have same size and format)
    '''
    imgs = iter(imgs)
    if PY3:
        img0 = next(imgs)
    else:
        img0 = imgs.next()
    pad = np.zeros_like(img0)
    imgs = it.chain([img0], imgs)
    rows = grouper(w, imgs, pad)
    return np.vstack(list(map(np.hstack, rows)))
def svmInit(C=12.5,gamma=0.50625):
    """
    创建并配置SVM模型
    :param C: 参数C
    :param gamma: 参数Gamma
    :return: 返回一个已经配置完参数的SVM模型
    """
    # 创建一个空SVM模型
    model = cv2.ml.SVM_create()
    model.setGamma(gamma)
    model.setC(C)
    # SVM核为SVM_RBF,需要有对应的Gamma参数
    model.setKernel(cv2.ml.SVM_RBF)
    # SVM类型为SVM_C_SVC，需要有对应的C参数
    model.setType(cv2.ml.SVM_C_SVC)
    return model
def svmPredict(model, samples):
    """
    返回模型对测试数据的预测结果
    :param model: 已经训练好的模型
    :param samples: 测试用的图像的特征向量
    :return: 返回模型对测试数据的预测结果
    """
    return model.predict(samples)[1].ravel()
def svmTrain(model, samples, labels):
    model.train(samples, cv2.ml.ROW_SAMPLE, labels)
    return model
def svmEvaluate(model, digits, samples, labels):    
    predictions = svmPredict(model, samples)
    accuracy = (labels == predictions).mean()
    print('预测准确率: %.2f %%' % (accuracy * 100))

    confusion = np.zeros((10, 10), np.int32)
    for i, j in zip(labels, predictions):
        confusion[int(i), int(j)] += 1
    print('confusion matrix:')
    print(confusion)
    print(sum(sum(confusion)))

    vis = []
    for img, flag in zip(digits, predictions == labels):
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        if not flag:
            img[..., :2] = 0
        vis.append(img)
    return mosaic(25, vis)


提取每张图片的haar特征向量


100%|██████████| 5000/5000 [01:16<00:00, 65.18it/s]


(5000, 12100) float32


In [1]:
print('定义HOG参数 ...')
# 定义HOG参数
hog = get_hog()
print('将数据集分成两份，90%用于训练，10%用于测试... ')
print('提取每张图片的HOG特征向量 ... ')
hog_descriptors = []
# 提取提取每张图像的特征向量，并添加到列表hog_descriptors
for img in digits_deskewed:
    hog_descriptors.append(hog.compute(img))
hog_descriptors = np.squeeze(hog_descriptors)
# Split the dataset correctly
train_n = int(0.9 * len(hog_descriptors))
digits_train, digits_test = np.split(digits_deskewed, [train_n])
hog_descriptors_train, hog_descriptors_test = np.split(hog_descriptors, [train_n])
labels_train, labels_test = np.split(labels, [train_n])
print('训练SVM模型 ...')
model = svmInit()
svmTrain(model, hog_descriptors_train, labels_train)
print('Evaluating model ... ')
vis = svmEvaluate(model, digits_test, hog_descriptors_test, labels_test)

定义HOG参数 ...


NameError: name 'get_hog' is not defined